In [74]:
import pandas as pd
import numpy as np

In [75]:
calendario = pd.read_csv("data/daily_calendar_with_events.csv")
precios = pd.read_csv("data/item_prices.csv")
ventas = pd.read_csv("data/item_sales.csv")

In [83]:
ventas1=ventas.copy()

In [86]:
ventasmelt=ventas.melt(id_vars=ventas.columns[:7],value_vars=ventas.columns[7:],var_name="d",value_name="Cantidad") #Transformacion de BD, dias por Fila

In [87]:
ventasmelt["Cantidad"].sum() # suma de ventas totales

65695409

In [90]:
ventas2=ventasmelt.drop(ventasmelt[ventasmelt["Cantidad"]==0].index) #eliminar indices con ventas =0

In [92]:
ventas2.reset_index(drop= True,inplace=True) # reset de indices despues de eliminar

In [93]:
ventas2["Cantidad"].sum() # comprobacion de suma total despues de eliminar filas check

65695409

In [94]:
calendario2=calendario.copy()

In [95]:
calendario2["date"]=pd.to_datetime(calendario2["date"]) # formato fecha en columna date

In [96]:
# crea columnas de dia,semana,mes y año a partir de date
calendario2['week'] = calendario2['date'].dt.isocalendar().week
calendario2['day'] = calendario2['date'].dt.day
calendario2['month'] = calendario2['date'].dt.month
calendario2['year'] = calendario2['date'].dt.year

In [97]:
# concatena semana y año en un string, agregando el 0 a semanas del 1 al 9
calendario2['week'] = calendario2['date'].dt.isocalendar().week.astype(str)
for i in range(len(calendario2)):
    if int(calendario2.loc[i,'week']) < 10:
        calendario2.loc[i,'week'] = '0'+calendario2.loc[i,'week']
calendario2['year'] = calendario2['date'].dt.isocalendar().year
calendario2['yearweek']=calendario2['year'].astype(str)+calendario2['week']

In [116]:
ventas3=pd.merge(ventas2,calendario2[["d","weekday_int","event","month","day","yearweek"]],how="left",on="d") # Join ventas con calendario

In [129]:
ventas3.shape

(18550276, 14)

In [136]:
ventas3.head(1)

,id,item,category,department,store,store_code,region,d,Cantidad,weekday_int,event,month,day,yearweek
0,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12,1,NaN,1,29,201104


In [146]:
columnasbase=ventas3.columns[:7].tolist()
columnasbase.append("yearweek")
columnasbase

['id',
 'item',
 'category',
 'department',
 'store',
 'store_code',
 'region',
 'yearweek']

In [157]:
ventas4=ventas3.groupby(by=columnasbase).agg({"Cantidad":"sum"})

In [160]:
ventas4=ventas4.reset_index()

In [162]:
ventas4["Cantidad"].sum()

65695409

In [163]:
ventas4.shape

(4990898, 9)

In [117]:
precios2=precios.copy()

In [118]:
precios["yearweek"].isnull().sum() # numero de semanas nulas

243920

In [119]:
precios["sell_price"].isnull().sum()

0

In [120]:
precios2["yearweek"]=precios2["yearweek"].astype(str)
precios2["yearweek"]=precios2["yearweek"].str.slice(stop=6) # quita .0 del string
precios2["year"]=precios2["yearweek"].str.slice(stop=4) # crea columna con año
precios2["semana"]=precios2["yearweek"].str.slice(start=4,stop=6) # crea columna con semana

In [121]:
precios2["id"]=precios2["item"]+"_"+precios2["store_code"] # crea id coditem+codstore

In [164]:
ventas5=pd.merge(ventas4,precios2[["id","yearweek","sell_price"]],how="left",on=["id","yearweek"]) #join de ventas y precios

In [165]:
ventas5.shape

(4990898, 10)

In [166]:
ventas5[ventas5["sell_price"].isnull()].index # ventas con datos nulos dado a que le semanaaño era nula

Index([     99,     231,     360,     498,     775,     917,     999,    1165,
          1866,    2223,
       ...
       4989876, 4990115, 4990211, 4990279, 4990373, 4990433, 4990506, 4990598,
       4990714, 4990774],
      dtype='int64', length=35126)

In [168]:
ventas_nulas=ventas5["Cantidad"].iloc[ventas5[ventas5["sell_price"].isnull()].index].sum() # cuantifica la cantidad de unidades con precio nulo

In [169]:
print((ventas_nulas/ventas_totales)*100,"%" )  #porcentaje de ventas nulas

0.510941335337451 %


In [170]:
ventas5.head(10)

,id,item,category,department,store,store_code,region,yearweek,Cantidad,sell_price
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201329,4,10.9858
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201330,4,10.9858
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201331,2,10.9858
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201332,1,10.9858
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201335,2,10.9858
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201336,3,10.9858
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201339,1,10.9858
7,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201341,2,10.9858
8,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201342,1,10.9858
9,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,201343,1,10.9858


In [172]:
from pathlib import Path  
filepath = Path("data/ventassem.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
ventas5.to_csv(filepath,index=False)  

In [173]:
calendario2.head()

,date,weekday,weekday_int,d,event,week,day,month,year,yearweek
0,2011-01-29,Saturday,1,d_1,NaN,04,29,1,2011,201104
1,2011-01-30,Sunday,2,d_2,NaN,04,30,1,2011,201104
2,2011-01-31,Monday,3,d_3,NaN,05,31,1,2011,201105
3,2011-02-01,Tuesday,4,d_4,NaN,05,1,2,2011,201105
4,2011-02-02,Wednesday,5,d_5,NaN,05,2,2,2011,201105


In [177]:
len(calendario2["yearweek"].unique())

274

In [174]:
from pathlib import Path  
filepath = Path("data/calendario.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
calendario2.to_csv(filepath,index=False)  

In [201]:
dep=ventas.groupby(["store_code","region","store","department"]).agg({"id":"count"})
dep=dep.reset_index()
del(dep["id"])
dep

,store_code,region,store,department
0,BOS_1,Boston,South_End,ACCESORIES_1
1,BOS_1,Boston,South_End,ACCESORIES_2
2,BOS_1,Boston,South_End,HOME_&_GARDEN_1
3,BOS_1,Boston,South_End,HOME_&_GARDEN_2
4,BOS_1,Boston,South_End,SUPERMARKET_1
...,...,...,...,...
65,PHI_3,Philadelphia,Queen_Village,HOME_&_GARDEN_1
66,PHI_3,Philadelphia,Queen_Village,HOME_&_GARDEN_2
67,PHI_3,Philadelphia,Queen_Village,SUPERMARKET_1
68,PHI_3,Philadelphia,Queen_Village,SUPERMARKET_2


In [202]:
from pathlib import Path  
filepath = Path("data/depto.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
dep.to_csv(filepath,index=False) 

In [204]:
prod=ventas.groupby(["category","item"]).agg({"id":"count"})
prod=prod.reset_index()
del(prod["id"])
prod

,category,item
0,ACCESORIES,ACCESORIES_1_001
1,ACCESORIES,ACCESORIES_1_002
2,ACCESORIES,ACCESORIES_1_003
3,ACCESORIES,ACCESORIES_1_004
4,ACCESORIES,ACCESORIES_1_005
...,...,...
3044,SUPERMARKET,SUPERMARKET_3_823
3045,SUPERMARKET,SUPERMARKET_3_824
3046,SUPERMARKET,SUPERMARKET_3_825
3047,SUPERMARKET,SUPERMARKET_3_826


In [ ]:
from pathlib import Path  
filepath = Path("data/productos.csv")  
filepath.parent.mkdir(parents=True, exist_ok=True)  
prod.to_csv(filepath,index=False) 